In [31]:
import mujoco
import glfw
import numpy as np
import time
from PIL import Image

XML_FILE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/irregular_shape_2D.xml"

def save_image_to_array(rgb_buffer, width, height, target_size):
    """
    Salva un'immagine dalla simulazione in un array ridimensionato alla dimensione target_size.
    """
    # Converti il buffer in un'immagine
    img = np.frombuffer(rgb_buffer, dtype=np.uint8).reshape(height, width, 3)
    img = Image.fromarray(img)

    # Ridimensiona l'immagine
    img_resized = img.resize(target_size, Image.LANCZOS)

    # Converti l'immagine ridimensionata in un array numpy
    return np.array(img_resized)

def main():

    dt = 0.01
    side_length = 0.6  # Lunghezza del lato del quadrato
    half_side = side_length / 2  # Metà lato (limiti del quadrato)
    alpha_x = 1
    alpha_y = 1
    beta_x = 1
    beta_y = 1
    std_x = 0.0    #std for gaussian noise on direction change and velocity. If 0.0 velocity is opposite
    std_y = 0.0    #std for gaussian noise on direction change and velocity. If 0.0 velocity is opposite

    if not glfw.init():
        print("Impossibile inizializzare GLFW")
        return

    glfw.window_hint(glfw.VISIBLE, glfw.TRUE)
    window = glfw.create_window(800, 800, "MuJoCo Viewer", None, None)
    if not window:
        print("Errore durante la creazione della finestra GLFW")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    model = mujoco.MjModel.from_xml_path(XML_FILE_PATH)
    data = mujoco.MjData(model)

    # Configura la telecamera
    camera = mujoco.MjvCamera()
    camera.type = mujoco.mjtCamera.mjCAMERA_FREE
    camera.lookat = np.array([0, 0, 0])
    camera.distance = 2.0
    camera.azimuth = 90
    camera.elevation = -90

    # Renderer per visualizzare la finestra
    scene = mujoco.MjvScene(model, maxgeom=1000)
    context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)

    t = 0  # Tempo iniziale
    frame_count = 0

    # Ottieni dimensioni del viewport
    viewport_width, viewport_height = glfw.get_framebuffer_size(window)

    # Prealloca buffer per RGB e profondità
    rgb_buffer = np.zeros((viewport_height, viewport_width, 3), dtype=np.uint8)
    depth_buffer = np.zeros((viewport_height, viewport_width), dtype=np.float32)

    frames = []

    while not glfw.window_should_close(window):

        t += dt

        # Posizione attuale
        x = data.qpos[0]
        y = data.qpos[1]

        # Rileva collisioni con i lati del quadrato
        if x >= half_side or x <= -half_side:
            alpha_x *= np.random.normal(-1, std_x)  # Inverti velocità lungo x
            print(f"Rimbalzo su lato verticale: x={x:.3f}, v_x={data.qvel[0]:.3f}")

        if y >= half_side or y <= -half_side:
            alpha_y *= np.random.normal(-1, std_y)  # Inverti velocità lungo y
            print(f"Rimbalzo su lato orizzontale: y={y:.3f}, v_y={data.qvel[1]:.3f}")

        # Calcola forze basate sulle velocità
        force_x = alpha_x * 1000 * beta_x * np.random.normal(1, 0.5)
        force_y = alpha_y * 1000 * beta_y * np.random.normal(1, 0.5)

        # Applica forze
        data.qfrc_applied[0] = force_x
        data.qfrc_applied[1] = force_y

        # Avanza la simulazione
        mujoco.mj_step(model, data)

        # Debug
        print(f"Posizione: x={x:.3f}, y={y:.3f}")
        print(f"Velocità: x_vel={data.qvel[0]:.3f}, y_vel={data.qvel[1]:.3f}")
        print(f"Forze: force_x={force_x:.3f}, force_y={force_y:.3f}")

        # Renderizza la scena
        mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
        mujoco.mjr_render(viewport, scene, context)

        # Salva il frame corrente come array
        frame_array = save_image_to_array(rgb_buffer.tobytes(), viewport_width, viewport_height, target_size=(64, 64))
        frames.append(frame_array)

        frame_count += 1

        glfw.swap_buffers(window)
        glfw.poll_events()
        time.sleep(dt)

    glfw.destroy_window(window)
    glfw.terminate()

    # Converti la lista di frame in un array numpy
    frames = np.stack(frames)
    print("Frames salvati per l'autoencoder:", frames.shape)

if __name__ == "__main__":
    main()


Posizione: x=0.000, y=0.000
Velocità: x_vel=0.171, y_vel=0.148
Forze: force_x=901.610, force_y=782.371
Posizione: x=0.002, y=0.001
Velocità: x_vel=0.322, y_vel=0.239
Forze: force_x=1716.770, force_y=1276.022
Posizione: x=0.005, y=0.004
Velocità: x_vel=0.081, y_vel=0.087
Forze: force_x=454.847, force_y=481.145
Posizione: x=0.006, y=0.005
Velocità: x_vel=0.083, y_vel=0.185
Forze: force_x=446.961, force_y=985.389
Posizione: x=0.007, y=0.007
Velocità: x_vel=0.370, y_vel=0.272
Forze: force_x=1959.968, force_y=1450.868
Posizione: x=0.010, y=0.009
Velocità: x_vel=0.157, y_vel=0.078
Forze: force_x=860.714, force_y=435.525
Posizione: x=0.012, y=0.010
Velocità: x_vel=0.208, y_vel=0.094
Forze: force_x=1113.796, force_y=502.066
Posizione: x=0.014, y=0.011
Velocità: x_vel=0.111, y_vel=0.189
Forze: force_x=605.240, force_y=1004.905
Posizione: x=0.015, y=0.013
Velocità: x_vel=0.209, y_vel=0.151
Forze: force_x=1488.475, force_y=1069.390
Posizione: x=0.017, y=0.014
Velocità: x_vel=0.062, y_vel=0.131
Fo

: 